## 任务3：语义检索Embedding模型

## M3E-small模型

In [ ]:
# 导入sentence_transformers
import json
import pdfplumber
from sentence_transformers import SentenceTransformer

In [ ]:
# 加载数据集，问题与知识源
questions = json.load(open("../data/Coggle比赛数据/汽车知识问答/questions.json",encoding='utf-8'))
pdf = pdfplumber.open("../data/Coggle比赛数据/汽车知识问答/初赛训练数据集.pdf")
pdf.pages
pdf.pages[0].extract_text()
# 此工具会将pdf分页，并提取每页的内容

In [ ]:
# 将pdf切分后的东西，将页码和内容组成json,并放在list中
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page':'page_'+str(page_idx+1),
        'content':pdf.pages[page_idx].extract_text()
    })
pdf_content[:1]

In [ ]:
# 解析question,与pdf中对应的内容（不包括page页码）到list中
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

pdf_content_sentences[-5]

In [6]:
# 加载embedding 模型

model = SentenceTransformer("moka-ai/m3e-small")

In [ ]:
# 将问题与pdf中的所有回答进行向量编码
question_embeddings = model.encode(question_sentences,normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences,normalize_embeddings=True)

In [ ]:
for query_idx,feat in enumerate(question_embeddings):
    score = feat@pdf_embeddings.T  # 将question中embedding后的每一个问题在pdf_embeddings中做相似度匹配，并打分
    max_score_page_idx = score.argsort()[-1]+1 #这是因为内容页的最后都会给出页码
    questions[query_idx]['reference']='page_'+str(max_score_page_idx)

questions[0]

In [9]:
with open('../data/Coggle比赛数据/汽车知识问答/submit_m3e.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

## BGE 模型

In [10]:
model = SentenceTransformer("BAAI/bge-small-zh-v1.5")

In [11]:
# 将问题与pdf中的所有回答进行向量编码
question_embeddings = model.encode(question_sentences,normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences,normalize_embeddings=True)

In [ ]:
for query_idx,feat in enumerate(question_embeddings):
    score = feat@pdf_embeddings.T  # 将question中embedding后的每一个问题在pdf_embeddings中做相似度匹配，并打分
    max_score_page_idx = score.argsort()[-1]+1 #这是因为内容页的最后都会给出页码
    questions[query_idx]['reference']='page_'+str(max_score_page_idx)

questions[0]

In [13]:
with open('../data/Coggle比赛数据/汽车知识问答/submit_bge.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

## BCEmbedding

In [14]:
model = SentenceTransformer("maidalun1020/bce-embedding-base_v1",device='cuda')
model.max_seq_length=512

In [15]:
# 将问题与pdf中的所有回答进行向量编码
question_embeddings = model.encode(question_sentences,normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences,normalize_embeddings=True)

In [ ]:
for query_idx,feat in enumerate(question_embeddings):
    score = feat@pdf_embeddings.T  # 将question中embedding后的每一个问题在pdf_embeddings中做相似度匹配，并打分
    max_score_page_idx = score.argsort()[-1]+1 #这是因为内容页的最后都会给出页码
    questions[query_idx]['reference']='page_'+str(max_score_page_idx)

questions[0]

In [17]:
with open('../data/Coggle比赛数据/汽车知识问答/submit_base_v1.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)